In [1]:
import h5py
import csv
import numpy as np
import glob
import gc  

def process_hdf5(h5name, output_merge, output_writer, is_first):
    print(f'Processing {h5name}...')
    
    with h5py.File(h5name, 'r') as dtfl:
        for evi in dtfl['data']:
            x = dtfl.get(f'data/{evi}')
            data = np.array(x)

            with h5py.File(output_merge, 'a') as HDFr:
                dsF = HDFr.create_dataset(f"data/{evi}", data.shape, data=data, dtype=np.float32)
                for attr in x.attrs:
                    dsF.attrs[attr] = x.attrs[attr]
            
            # Запись в CSV
            output_writer.writerow([x.attrs[attr] for attr in x.attrs])

            # Освобождение памяти
            del data, dsF, x
            gc.collect()

def main(inp_name):
    output_merge = inp_name + '.hdf5'
    
    with h5py.File(output_merge, 'w') as HDF0:
        HDF0.create_group("data")
    
    csv_filename = output_merge.replace('.hdf5', '.csv')
    with open(csv_filename, 'w', newline='') as csvfile:
        output_writer = csv.writer(csvfile)
        
        sample_h5 = glob.glob('chunk*.hdf5')[0]
        with h5py.File(sample_h5, 'r') as sample_file:
            sample_event = list(sample_file['data'].keys())[0]
            sample_attrs = list(sample_file[f'data/{sample_event}'].attrs.keys())
            output_writer.writerow(sample_attrs)
        
        # Processing HDF5 files
        h5_files = glob.glob('chunk*.hdf5')
        if not h5_files:
            print("Не найдено файлов 'chunk*.hdf5'. Завершение программы.")
        else:
            print(f"Найдено {len(h5_files)} файлов для обработки.")
        
        for i, h5name in enumerate(h5_files):
            print(f"Обрабатывается файл: {h5name}")  # Вывод имени текущего файла
            process_hdf5(h5name, output_merge, output_writer, is_first=(i == 0))

    print('Processing complete!')

if __name__ == '__main__':
    main("merge1")


Найдено 6 файлов для обработки.
Обрабатывается файл: chunk1.hdf5
Processing chunk1.hdf5...
Обрабатывается файл: chunk2.hdf5
Processing chunk2.hdf5...
Обрабатывается файл: chunk3.hdf5
Processing chunk3.hdf5...
Обрабатывается файл: chunk4.hdf5
Processing chunk4.hdf5...
Обрабатывается файл: chunk5.hdf5
Processing chunk5.hdf5...
Обрабатывается файл: chunk6.hdf5
Processing chunk6.hdf5...
Processing complete!
